# TESTING DOWNLOAD BASED ON FILTERED DATASET (AFTER QUERY AND USER CHECK)
## Using ChatGPT to code this

In [1]:
import requests
import pandas as pd
import os


In [2]:
df_recordings = pd.read_csv('/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/tables/filtered_df.csv')

In [3]:
df_recordings[:3].head()

,Unnamed: 0,id,rec,loc,gen,sp,lat,lng,alt,type,q,length,bird-seen,file,en
0,0,162251,Mike Nelson,"Berenty Reserve, Toliara Prov.",Coua,gigas,-25.0590,46.3028,160,song,B,9:06,yes,https://xeno-canto.org/162251/download,Giant Coua
1,1,162878,Mike Nelson,"Ranomafana National Park, Fianarantsoa",Coua,caerulea,-21.4326,47.3978,1100,alarm call,B,3:06,yes,https://xeno-canto.org/162878/download,Blue Coua
2,2,68027,Frank Lambert,Zombitse Forest,Coua,gigas,-22.9000,44.7000,800,"call, song",A,2:56,unknown,https://xeno-canto.org/68027/download,Giant Coua


In [10]:
for index, row in df_recordings[:3].iterrows():
    gen = row['gen']
    sp = row['sp']
    ID = row['id']
    URL = row['file']
    
    # Create directory if it doesn't exist
    directory = gen + '_' + sp
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    # Download file
    response = requests.get(URL)
    open(directory + '/' + str(ID) + '.wav', 'wb').write(response.content)

In [11]:
import os
import requests

for index, row in df_recordings[:3].iterrows():
    directory = row["gen"] + '_' + row["sp"]
    os.makedirs(directory, exist_ok=True)
    open(directory + '/' + str(row["id"]) + '.wav', 'wb').write(requests.get(row["file"]).content)


In [12]:
import os
import requests

for index, row in df_recordings.iterrows():
    directory = row["gen"] + '_' + row["sp"]
    os.makedirs(directory, exist_ok=True)
    file_path = directory + '/' + str(row["id"]) + '.wav'
    try:
        response = requests.get(row["file"], stream=True, timeout=5)
        response.raise_for_status()
        with open(file_path, 'wb') as f:
            for chunk in response.iter_content(1024):
                f.write(chunk)
    except (requests.exceptions.RequestException, IOError) as e:
        if os.path.exists(file_path):
            os.remove(file_path)
        print("An error occurred while processing: " + row["file"])
        print("Error: ", e)


In [5]:
import os
import requests
from tqdm import tqdm

overwrite=True
for index, row in df_recordings[:3].iterrows():
    directory = row["gen"] + '_' + row["sp"]
    os.makedirs(directory, exist_ok=True)
    file_path = directory + '/' + str(row["id"]) + '.wav'
    if os.path.exists(file_path) and not overwrite:
        continue
    try:
        response = requests.get(row["file"], stream=True, timeout=5)
        response.raise_for_status()
        with open(file_path, "wb") as f:
            total_size = int(response.headers.get("content-length", 0))
            for data in tqdm(response.iter_content(1024), total=total_size, unit='B', unit_scale=True, desc=file_path, leave=False):
                f.write(data)
    except (requests.exceptions.RequestException, IOError) as e:
        if os.path.exists(file_path):
            os.remove(file_path)
        print("An error occurred while processing: " + row["file"])
        print("Error: ", e)


### SINGLE DOWNLOAD

In [6]:
import os
import requests
from tqdm import tqdm

def download_files(df_recordings, root_dir='.', overwrite=True):
    for index, row in df_recordings.iterrows():
        directory = os.path.join(root_dir, row["gen"] + '_' + row["sp"])
        os.makedirs(directory, exist_ok=True)
        file_path = os.path.join(directory, str(row["id"]) + '.wav')
        if os.path.exists(file_path) and not overwrite:
            continue
        try:
            response = requests.get(row["file"], stream=True, timeout=5)
            response.raise_for_status()
            with open(file_path, "wb") as f:
                total_size = int(response.headers.get("content-length", 0))
                for data in tqdm(response.iter_content(1024), total=total_size, unit='B', unit_scale=True, desc=file_path, leave=False):
                    f.write(data)
        except (requests.exceptions.RequestException, IOError) as e:
            if os.path.exists(file_path):
                os.remove(file_path)
            print("An error occurred while processing: " + row["file"])
            print("Error: ", e)


In [7]:
download_files(df_recordings=df_recordings,root_dir='/Users/Paul/Paul/Desktop/My_projects/Bioacoustics/Maputo_Dash/datasets/wav/TEST')

An error occurred while processing: https://xeno-canto.org/162251/download
Error:  HTTPSConnectionPool(host='xeno-canto.org', port=443): Read timed out.


An error occurred while processing: https://xeno-canto.org/26705/download
Error:  500 Server Error: Internal Server Error for url: https://xeno-canto.org/26705/download


An error occurred while processing: https://xeno-canto.org/26704/download
Error:  500 Server Error: Internal Server Error for url: https://xeno-canto.org/26704/download


An error occurred while processing: https://xeno-canto.org/26702/download
Error:  500 Server Error: Internal Server Error for url: https://xeno-canto.org/26702/download


An error occurred while processing: https://xeno-canto.org/26703/download
Error:  500 Server Error: Internal Server Error for url: https://xeno-canto.org/26703/download


### MULTICORE DOWNLOAD

In [ ]:
import os
import requests
from tqdm import tqdm
from concurrent.futures import ProcessPoolExecutor

def download_file(file_path, url, overwrite):
    if os.path.exists(file_path) and not overwrite:
        return
    try:
        response = requests.get(url, stream=True, timeout=5)
        response.raise_for_status()
        with open(file_path, "wb") as f:
            total_size = int(response.headers.get("content-length", 0))
            for data in tqdm(response.iter_content(1024), total=total_size, unit='B', unit_scale=True, desc=file_path, leave=False):
                f.write(data)
    except (requests.exceptions.RequestException, IOError) as e:
        if os.path.exists(file_path):
            os.remove(file_path)
        print("An error occurred while processing: " + url)
        print("Error: ", e)

def download_files(df_recordings, root_dir='.', overwrite=True):
    with ProcessPoolExecutor(max_workers=2) as executor:
        futures = []
        for index, row in df_recordings.iterrows():
            directory = os.path.join(root_dir, row["gen"] + '_' + row["sp"])
            os.makedirs(directory, exist_ok=True)
            file_path = os.path.join(directory, str(row["id"]) + '.wav')
            futures.append(executor.submit(download_file, file_path, row["file"], overwrite))
        for future in futures:
            future.result()
